## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):


### Confidence Intervals:
1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

### Chi-squared tests:
4. Take a dataset that we have used in the past in class that has **categorical** variables. Pick two of those categorical variables and run a chi-squared tests on that data
  - By hand using Numpy
  - In a single line using Scipy


In [0]:
# Imports
from statistics import *
import pandas as pd
import numpy as np
from scipy.stats import t, ttest_ind

In [0]:
# Specify the columns for the UCI dataset
cols = [
    'party',
    'handicapped-infants',
    'water-project',
    'budget',
    'physician-fee-freeze',
    'el-salvador-aid',
    'religious-groups',
    'anti-satellite-ban',
    'aid-to-contras',
    'mx-missile',
    'immigration',
    'synfuels',
    'education',
    'right-to-sue',
    'crime',
    'duty-free',
    'south-africa',
    ]


# Assign the url string to a variable
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

# Fetch the csv data assuming no header row in the data"
#   1. assign the column names specified above
#   2. replace '?' with NaN values
df_house = pd.read_csv(url, header=None, names=cols, na_values='?')

# Translate 'y' and 'n' votes to numeric '1' and'0'
df_house = df_house.replace({'y': 1, 'n': 0})


In [0]:
# sample_me returns a list of means of house vote samples for use downstream
#    issue: the issue being voted on
#    num: number of samples to be taken
#    size: sample size (# of votes in sample)
def sample_me(issue, num, size):
  # Set up working variables
  rt_list = [] # return a list of means

  # Generate a series of the issue's votes
  srs_tmp = df_house[issue]
  # Drop nas from the series votes
  srs_tmp = srs_tmp.dropna()

  # Number of votes on this issue
  tot_vts = len(srs_tmp)

  # Iterate to create 'num' sample means
  for i in range(1, num):
    # Randomly select a sample of 'size'
    smpl_iter = srs_tmp.sample(n=size)

    # Generate the mean of the sample
    tmp_mean = smpl_iter.mean()

    # Add the mean to our return list
    rt_list.append(tmp_mean)
  
  # Return the temp series and list of sample means
  return {"means": rt_list, "series": srs_tmp}
   

In [0]:
def gen_conf_intvls(issue):
  # Generate 10 samples of 20 votes of issue 'issue'
  rslts_sample = sample_me(issue, 10, 20)

  tmp_means = rslts_sample["means"]
  # Determine the sample size
  n = len(tmp_means)

  # Determine the 'degrees of freedom'
  dof = n - 1

  # Calculate the mean of the sample means
  sample_mean = mean(tmp_means)

  # Calculate the standard deviation of the sample means
  sample_std = np.std(tmp_means, ddof=1)

  # Calclate the standard error of the sample means
  sample_std_err = sample_std/(n**.5)

  # Generate the confidence interval
  conf_int = t.interval(.95, dof, loc=sample_mean, scale=sample_std_err)

  # Return output
  return {"lower": conf_int[0], "mean": sample_mean, "upper": conf_int[1], "list_means": tmp_means}


In [0]:
# Iterate through our issues
lst_datfrm = []     # list of issue confidence intervals

for i in range(1, len(cols)):
  # Generate the confidence interval 
  tmp_rslts = gen_conf_intvls(cols[i])

  # Construct a dataframe row for use downstream

  # Start with adding the issue, CI lower bound, mean, and CI upper bound
  tmp_row = [cols[i], tmp_rslts["lower"], tmp_rslts["mean"], tmp_rslts["upper"]]

  # Then add the sample means
  for i in range(0, len(tmp_rslts["list_means"])):
    tmp_row.append(tmp_rslts["list_means"][i])

  # Append to a working list of rows which will (ultimately) make up our dataframe
  lst_datfrm.append(tmp_row)


In [0]:
# Create a dataframe of confidence interval results
df_cols = ['Issue', 'LowerBound', 'Mean', 'UpperBound']
for i in range(1, 20):
  df_cols.append(i)

df_issue_CIs = pd.DataFrame(lst_datfrm, columns=df_cols)
df_issue_CIs

## Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)